In [1]:
import cv2
import csv

In [2]:
filenames = []
labels = []
with open('Labels.csv') as index:
    index = csv.reader(index)
    for r in index:
        filenames.append(r[0])
        labels.append(r[1])
        
del filenames[0]
del labels[0]

In [3]:
import numpy as np

In [4]:
labels_hot = []
for x in labels:
    if x == 'Positive':
        labels_hot.append([1,0,0])
    if x == 'Neutral':
        labels_hot.append([0,1,0])
    if x == 'Negative':
        labels_hot.append([0,0,1])

In [5]:
lb_idx = 0
final_label=[]
f_idx = 0
classfier = cv2.CascadeClassifier('haarcascade_frontalface_alt2.xml')
color = (0,255,0)
for file in filenames:
    filename = './presidential_videos/' + file
    cap = cv2.VideoCapture(filename)
    count = 0
    while(cap.isOpened()):
        success, frame = cap.read()
        if not success:
            break
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        faceRects = classfier.detectMultiScale(gray, scaleFactor=1.5,minNeighbors=8, minSize=(32,32))
        
        if len(faceRects)>0:
            for fr in faceRects:
                x,y,w,h = fr
#                 cv2.rectangle(frame,(x-20,y-20), (x+w+10,y+h+10), color, 1)
                image = frame[y:y+h, x:x+w]
                im_name = './data/' + str(f_idx)+'.jpg'
                final_label.append(labels[lb_idx])
                f_idx += 1
                cv2.imwrite(im_name, image)
                count += 1
#                 cv2.imshow('frame', frame)
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break
        if count >= 30:
            break
    lb_idx += 1
#     break
# cv2.destroyAllWindows()
cap.release()

In [6]:
print(len(final_label))

27499


In [7]:
import os

image_size = 64
def resize_img(image, height = image_size, width = image_size):
    top,bottom, left, right = (0,0,0,0)
    
    h,w,_=image.shape
    
    longest_edge = max(h,w)
    
    if h < longest_edge:
        dh = longest_edge - h
        top = dh // 2
        bottom = dh - top
    elif w < longest_edge:
        dw = longest_edge - w
        left = dw // 2
        right = dw - left
    else:
        pass
    
    BLACK = [0,0,0]
    constant = cv2.copyMakeBorder(image, top,bottom, left,right, cv2.BORDER_CONSTANT,value = BLACK)
    
    return cv2.resize(constant, (height,width))

In [8]:
images = []


for i in range(len(final_label)):
    img_name = str(i) +'.jpg'
    image = cv2.imread('./data/'+img_name)
    if image is None:
        del final_label[i]
    else:
        image = resize_img(image, image_size, image_size)
        images.append(image)
        
print(len(images))
print(len(final_label))
        

27499
27499


In [9]:
images = np.array(images, dtype='float')
final_label = np.array(final_label)

print(images.shape)
print(final_label.shape)



(27499, 64, 64, 3)
(27499,)


In [15]:
import keras
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.models import load_model
from keras import backend as K

In [17]:
import random
train_images, test_images, train_labels, test_labels = train_test_split(images, final_label, test_size = 0.3, random_state = random.randint(0,100))
print(test_labels)

['Neutral' 'Positive' 'Neutral' ... 'Neutral' 'Neutral' 'Positive']


In [22]:
img_channels = 3

In [25]:

if K.image_data_format == 'channel_first':
    train_images = train_images.reshape(train_images.shape[0],img_channels, image_size, image_size)
    test_images = test_images.reshape(test_images.shape[0],img_channels, image_size, image_size)
    input_shape = (img_channels, image_size, image_size)
else:
    train_images = train_images.reshape(train_images.shape[0], image_size, image_size, img_channels)
    test_images = test_images.reshape(test_images.shape[0], image_size, image_size, img_channels)
    input_shape = (image_size, image_size, img_channels)

In [26]:
print(train_images.shape[0],'train samples')
print(test_images.shape[0], 'test samples')

19249 train samples
8250 test samples


In [27]:
train_images /= 255
test_images /= 255

In [ ]:
train_labels = train_labels.tolist()
test_labels = test_labels.tolist()

In [40]:
train_labels_hot = []
test_labels_hot = []
for x in train_labels:
    if x == 'Positive':
        train_labels_hot.append([1,0,0])
    if x == 'Neutral':
        train_labels_hot.append([0,1,0])
    if x == 'Negative':
        train_labels_hot.append([0,0,1])
        
for x in test_labels:
    if x == 'Positive':
        test_labels_hot.append([1,0,0])
    if x == 'Neutral':
        test_labels_hot.append([0,1,0])
    if x == 'Negative':
        test_labels_hot.append([0,0,1])

In [64]:
np.savetxt('final_label.csv',final_label, delimiter=',',fmt = '%s')

In [65]:
test = []
with open('final_label.csv') as a:
    a = csv.reader(a)
    for r in a:
        test.append(r)

In [72]:
class Model:
    def __init__(self):
        self.model = None
    def build_model(self, input_shape, nb_classes=3):
        self.model = Sequential()
        self.model.add(Conv2D(32, (3, 3), padding = 'same', input_shape = input_shape))
        self.model.add(Activation('relu'))
        self.model.add(Conv2D(32, (3, 3)))
        self.model.add(Activation('relu'))
        self.model.add(MaxPooling2D(pool_size = (2,2)))
        self.model.add(Conv2D(64, (3, 3), padding = 'same'))
        self.model.add(Activation('relu'))
        self.model.add(Conv2D(64, (3, 3)))
        self.model.add(Activation('relu'))
        self.model.add(MaxPooling2D(pool_size = (2,2)))
        self.model.add(Dropout(0.25))
        self.model.add(Flatten())
        self.model.add(Dense(512))
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.25))
        self.model.add(Dense(nb_classes))
        self.model.add(Activation('softmax'))
        
    def train(self, train_images, train_labels, batch_size = 128, nb_epoch = 15, data_augmentation = True):
        self.model.compile(loss = 'categorical_crossentropy', 
                           optimizer = 'ADAM',
                           metrics = ['accuracy'])
        if not data_augmentation:
            self.model.fit(train_images, 
                           train_labels, 
                           batch_size = batch_size,
                           epochs = nb_epoch, 
                           shuffle = True)
        else:
            datagen = ImageDataGenerator(rotation_range = 20,
                                        width_shift_range = 0.2,
                                        height_shift_range = 0.2,
                                        horizontal_flip = True)
            
            self.model.fit_generator(datagen.flow(train_images, train_labels, batch_size = batch_size), epochs = nb_epoch)
            
    def evaluate(sefl, test_images, test_labels):
        score = self.model.evaluate(test_images, test_labels)
        print("%s:%.3f%%" % (self.model.metrics_names[1], score[1]*100))
        

In [73]:
model = Model()

In [74]:
model.build_model(input_shape,3)

In [80]:
np.array(train_labels_hot)
np.array(test_labels_hot)

array([[0, 1, 0],
       [1, 0, 0],
       [0, 1, 0],
       ...,
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0]])

In [ ]:
model.train(train_images, train_labels_hot, 128,30,True)

Epoch 1/30
151/151 [==============================] - 127s 841ms/step - loss: 0.9517 - accuracy: 0.5440
Epoch 2/30
151/151 [==============================] - 128s 849ms/step - loss: 0.9186 - accuracy: 0.5665
Epoch 3/30
151/151 [==============================] - 128s 850ms/step - loss: 0.8863 - accuracy: 0.5881
Epoch 4/30
151/151 [==============================] - 136s 904ms/step - loss: 0.8562 - accuracy: 0.5990
Epoch 5/30
151/151 [==============================] - 131s 866ms/step - loss: 0.8293 - accuracy: 0.6143
Epoch 6/30
151/151 [==============================] - 126s 834ms/step - loss: 0.8099 - accuracy: 0.6237
Epoch 7/30
151/151 [==============================] - 127s 842ms/step - loss: 0.7911 - accuracy: 0.6374
Epoch 8/30
151/151 [==============================] - 125s 827ms/step - loss: 0.7723 - accuracy: 0.6457
Epoch 9/30
151/151 [==============================] - 130s 859ms/step - loss: 0.7557 - accuracy: 0.6540
Epoch 10/30
151/151 [==============================] - 129s 856m